# Zillow Property Search


## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [ ]:
from google.colab import output, drive, files # specific to Google Colab
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import warnings

# settings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

## <font color="blue">Functions</font>

In [ ]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

## <font color="blue">Locals & Constants</font>

In [ ]:
############
# OPTIONAL #
############

# mount drive
drive.mount("/content/drive", force_remount=False)

# data location
file_dir = "/content/drive/My Drive/Colab Data/input/" # optional

Mounted at /content/drive


In [ ]:
# read in api key file
# df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
# api_key = df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

api_key = '...'

## <font color="blue">Data</font>

### <font color="green">1. Property Listings For Sale</font>

In [ ]:
# zillow search url
listing_url = "https://www.zillow.com/tampa-fl/houses/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.96171153027345%2C%22east%22%3A-81.94684946972657%2C%22south%22%3A27.709897826482354%2C%22north%22%3A28.2786076346999%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A200000%2C%22max%22%3A550000%7D%2C%22mp%22%3A%7B%22min%22%3A946%2C%22max%22%3A2366%7D%2C%22beds%22%3A%7B%22min%22%3A2%7D%2C%22built%22%3A%7B%22min%22%3A1940%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"

# get listings
listing_response = get_listings(api_key, listing_url)

In [ ]:
# view all keys
listing_response.json().keys()

dict_keys(['is_success', 'data', 'message'])

In [ ]:
# check if request is successful
listing_response.json()["is_success"]

True

In [ ]:
# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 311


In [ ]:
# view all listings
df_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

Number of rows: 304
Number of columns: 94


,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,beds,baths,area,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,info3String,brokerName,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,variableData.type,variableData.text,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,hdpData.homeInfo.videoCount,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,hdpData.homeInfo.unit,timeOnZillow,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.listing_sub_type.is_forAuction,minBeds,minBaths,minArea,buildingId,isBuilding,canSaveBuilding,unitCount,builderName,plid,communityName,isCdpResult,isComingSoonCommunity,lotId,style
0,45076178,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/bb5837cb06a...,True,/homedetails/4605-Regnas-Ave-Tampa-FL-33617/45...,FOR_SALE,House for sale,"$383,000",$383K,"4605 Regnas Ave, Tampa, FL 33617",4.0,3.0,1987.0,False,False,ForSale,For Sale (Broker),False,False,False,False,True,False,False,,False,False,https://photos.zillowstatic.com/fp/e39375731b4...,REALTY ONE GROUP ADVANTAGE,28.023796,-82.409134,4.507618e+07,4605 Regnas Ave,33617,Tampa,FL,28.023796,-82.409134,383000.0,3.0,4.0,1987.0,SINGLE_FAMILY,FOR_SALE,-1.0,False,False,383200.0,2515.0,True,False,False,FOR_SALE,383000.0,True,False,False,USD,USA,374450.0,0.886846,acres,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2062577189,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/250a04ec8eb...,True,/homedetails/7222-Spring-Snowflake-Ave-Tampa-F...,FOR_SALE,House for sale,"$435,000",$435K,"7222 Spring Snowflake Ave, Tampa, FL 33619",4.0,3.0,2073.0,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,False,,False,False,https://photos.zillowstatic.com/fp/e39375731b4...,SIGNATURE REALTY ASSOCIATES,27.927029,-82.379990,2.062577e+09,7222 Spring Snowflake Ave,33619,Tampa,FL,27.927029,-82.379990,435000.0,3.0,4.0,2073.0,SINGLE_FAMILY,FOR_SALE,-1.0,False,False,NaN,NaN,True,False,False,FOR_SALE,435000.0,True,False,False,USD,USA,NaN,3719.000000,sqft,False,3D_HOME,3D Tour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,45078919,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/93dddf13e04...,True,/homedetails/8523-N-Mulberry-St-Tampa-FL-33604...,FOR_SALE,House for sale,"$265,000",$265K,"8523 N Mulberry St, Tampa, FL 33604",3.0,2.0,1206.0,False,False,ForSale,For Sale (Broker),False,False,False,False,True,False,False,,False,False,https://photos.zillowstatic.com/fp/e39375731b4...,HERBERT R. FISHER REALTY,28.028828,-82.

In [ ]:
# view price
px.histogram(df_listings, x="hdpData.homeInfo.price", title="Sales Price Histogram")

In [ ]:
# view zestimate
px.histogram(df_listings, x="hdpData.homeInfo.zestimate", title="Zestimate Histogram")

In [ ]:
# view rent zestimate
px.box(df_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")

### <font color="green">2. Rental Property Listings For Rent</font>

In [ ]:
# zillow search url
rent_listing_url = "https://www.zillow.com/tampa-fl/rentals/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.69632304150392%2C%22east%22%3A-82.2122379584961%2C%22south%22%3A27.743327003864252%2C%22north%22%3A28.2453440906427%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priorityscore%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22beds%22%3A%7B%22min%22%3A3%2C%22max%22%3A3%7D%2C%22baths%22%3A%7B%22min%22%3A2%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D"

# get listings
rent_listing_response = get_listings(api_key, rent_listing_url)

In [ ]:
# view all keys
print(rent_listing_response.json().keys())

# check if request is successful
print("Request success:", rent_listing_response.json()["is_success"])

# view count of properies returned in request
num_of_properties = rent_listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

dict_keys(['is_success', 'data', 'message'])
Request success: True
Count of properties: 264


In [ ]:
# view all listings
df_rent_listings = pd.json_normalize(rent_listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_rent_listings))
print("Number of columns:", len(df_rent_listings.columns))
df_rent_listings

Number of rows: 262
Number of columns: 77


,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,beds,baths,area,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,availabilityDate,visited,timeOnZillow,rentalMarketingSubType,latLong.latitude,latLong.longitude,variableData.type,variableData.text,variableData.data.isRead,variableData.data.isFresh,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.zestimate,hdpData.homeInfo.videoCount,hdpData.homeInfo.unit,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceChange,hdpData.homeInfo.priceReduction,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,streetViewURL,streetViewMetadataURL
0,299381288,ForRent,For Rent,https://photos.zillowstatic.com/fp/350a410f67d...,True,/homedetails/2702-N-Morgan-St-Tampa-FL-33602/2...,FOR_RENT,House for rent,"$4,195/mo",$4.2K,"2702 N Morgan St, Tampa, FL 33602",3,2.5,1356.0,False,False,ForRent,For Rent,False,False,False,False,False,True,False,,False,2024-05-15 00:00:00,False,53139000,unknown,27.967272,-82.458206,TIME_ON_INFO,14 hours ago,NaN,True,299381288,2702 N Morgan St,33602,Tampa,FL,27.967272,-82.458206,4195.0,2.5,3.0,1356.0,SINGLE_FAMILY,FOR_RENT,-1,True,False,False,1814.0,False,False,FOR_RENT,4195.0,53139000,True,False,False,USD,USA,23512.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45055621,ForRent,For Rent,https://photos.zillowstatic.com/fp/ec80a79fae5...,True,/homedetails/3623-S-Lightner-Dr-Tampa-FL-33629...,FOR_RENT,House for rent,"$3,600/mo",$3.6K,"3623 S Lightner Dr, Tampa, FL 33629",3,2.0,1455.0,False,False,ForRent,For Rent,False,False,False,False,False,True,False,,False,2024-06-01 00:00:00,False,81968000,unknown,27.909273,-82.528800,TIME_ON_INFO,22 hours ago,NaN,True,45055621,3623 S Lightner Dr,33629,Tampa,FL,27.909273,-82.528800,3600.0,2.0,3.0,1455.0,SINGLE_FAMILY,FOR_RENT,-1,True,False,False,3025.0,False,False,FOR_RENT,3600.0,81968000,True,False,False,USD,USA,471340.0,False,686200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,45088364,ForRent,For Rent,https://photos.zillowstatic.com/fp/ce1803f9476...,True,/homedetails/1701-E-New-Orleans-Ave-Tampa-FL-3...,FOR_RENT,House for rent,"$2,100/mo",$2.1K,"1701 E New Orleans Ave, Tampa, FL 33610",3,2.0,851.0,False,False,ForRent,For Rent,False,False,False,False,False,True,False,,False,2024-04-07 00:00:00,False,149960000,unknown,27.990004,-82.440780,TIME_ON_INFO,1 day ago,NaN,False,45088364,1701 E New Orleans Ave,33610,Tampa,FL,27.990004,-82.440780,2100.0,2.0,3.0,851.0,SINGLE_FAMILY,FOR_RENT,-1,True,False,False,1949.0,False,False,FOR_RENT,2100.0,149960000,True,False,False,USD,USA,109208.0,False,280400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,45104543,ForRent,For Rent,https://photos.zillowstatic.com/fp/3093940ab79...,True,/homedetails/2902-W-Nassau-St-Tampa-FL-33607/4...,FOR_RENT,House for rent,"$3,990/mo",$4.0K,"2902 W Nassau St, Tampa, FL 33607",3,2.0,1319.0,False,False,ForRent,For Rent,False,False,False,False,False,True,False,,False,2024-04-07 00:00:00,False,201438000,unknown,27.953373,-82

In [ ]:
# view price
px.box(df_rent_listings, x="hdpData.homeInfo.price", title="Rental Price Box Plot")

In [ ]:
# view rent zestimate
px.box(df_rent_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")

### <font color="green">3. Property Details</font>

In [ ]:
# property unique id
zpid = "99616959"

# get property detail
prop_detail_response = get_property_detail(api_key, zpid)

In [ ]:
# view all keys
print(prop_detail_response.json().keys())

# check if request is successful
print("Request success:", prop_detail_response.json()["is_success"])

dict_keys(['is_success', 'data', 'message'])
Request success: True


In [ ]:
# view property details
df_prop = pd.json_normalize(prop_detail_response.json()['data'])
print("Number of rows:", len(df_prop))
print("Number of columns:", len(df_prop.columns))
df_prop

Number of rows: 1
Number of columns: 603


,listingDataSource,zpid,city,state,homeStatus,isListingClaimedByCurrentSignedInUser,isCurrentSignedInAgentResponsible,bedrooms,bathrooms,price,yearBuilt,streetAddress,zipcode,isCurrentSignedInUserVerifiedOwner,regionString,propertyUpdatePageLink,moveHomeMapLocationLink,propertyEventLogLink,editPropertyHistorylink,isRentalListingOffMarket,hdpUrl,nearbyCities,nearbyNeighborhoods,country,nearbyZipcodes,abbreviatedAddress,building,isUndisclosedAddress,boroughId,providerListingID,boroughSearchUrl,communityUrl,isPremierBuilder,isZillowOwned,homeType,currency,listPriceLow,livingArea,livingAreaValue,zestimate,newConstructionType,zestimateLowPercent,zestimateHighPercent,rentZestimate,restimateLowPercent,restimateHighPercent,schools,homeValues,nearbyHomes,countyFIPS,parcelId,taxHistory,priceHistory,comps,description,whatILove,contingentListingType,timeOnZillow,pageViewCount,favoriteCount,daysOnZillow,latitude,longitude,openHouseSchedule,desktopWebHdpImageLink,brokerageName,timeZone,pals,listingAccountUserId,homeInsights,sellingSoon,listingProvider,isIncomeRestricted,brokerId,ssid,monthlyHoaFee,propertyTaxRate,hiResImageLink,hdpTypeDimension,mlsid,propertyTypeDimension,mediumImageLink,enhancedBrokerImageUrl,responsivePhotos,buildingId,virtualTourUrl,hasApprovedThirdPartyVirtualTourUrl,photoCount,livingAreaUnits,lotSize,lotAreaValue,lotAreaUnits,postingProductType,marketingName,richMedia,cityId,stateId,zipPlusFour,numberOfUnitsTotal,foreclosureDefaultFilingDate,foreclosureAuctionFilingDate,foreclosureLoanDate,foreclosureLoanOriginator,foreclosureLoanAmount,foreclosurePriorSaleDate,foreclosurePriorSaleAmount,foreclosureBalanceReportingDate,foreclosurePastDueBalance,foreclosureUnpaidBalance,foreclosureAuctionTime,foreclosureAuctionDescription,foreclosureAuctionCity,foreclosureAuctionLocation,foreclosureDate,foreclosureAmount,foreclosingBank,foreclosureJudicialType,datePostedString,foreclosureMoreInfo,hasBadGeocode,streetViewMetadataUrlMediaWallLatLong,streetViewMetadataUrlMediaWallAddress,streetViewTileImageUrlMediumLatLong,streetViewTileImageUrlMediumAddress,streetViewServiceUrl,postingUrl,hasPublicVideo,primaryPublicVideo,richMediaVideos,photos,tourViewCount,listingAccount,listingFeedID,livingAreaUnitsShort,priceChange,priceChangeDate,priceChangeDateString,hideZestimate,comingSoonOnMarketDate,isPreforeclosureAuction,lastSoldPrice,isHousingConnector,responsivePhotosOriginalRatio,streetViewMetadataUrlMapLightboxAddress,thumb,isRecentStatusChange,isNonOwnerOccupied,county,isFeatured,rentalApplicationsAcceptedType,listingTypeDimension,featuredListingTypeDimension,brokerIdDimension,keystoneHomeStatus,pageUrlFragment,isRentalsLeadCapMet,isPaidMultiFamilyBrokerId,address.streetAddress,address.city,address.state,address.zipcode,address.neighborhood,address.community,address.subdivision,listing_sub_type.is_newHome,listing_sub_type.is_FSBO,listing_sub_type.is_FSBA,listing_sub_type.is_foreclosure,listing_sub_type.is_bankOwned,listing_sub_type.is_forAuction,listing_sub_type.is_comingSoon,listing_sub_type.is_pending,listing_sub_type.is_openHouse,citySearchUrl.text,citySearchUrl.path,zipcodeSearchUrl.path,apartmentsForRentInZipcodeSearchUrl.path,housesForRentInZipcodeSearchUrl.path,neighborhoodRegion.name,neighborhoodSearchUrl.path,stateSearchUrl.path,countySearchUrl.text,countySearchUrl.path,adTargets.aamgnrc1,adTargets.bd,adTargets.fsbid,adTargets.city,adTargets.proptp,adTargets.pid,adTargets.lot,adTargets.zestibuck,adTargets.listtp,adTargets.sqftrange,adTargets.price,adTargets.sqft,adTargets.state,adTargets.mlong,adTargets.cnty,adTargets.prange,adTargets.zip,adTargets.mlat,adTargets.zusr,adTargets.price_band,adTargets.yrblt,adTargets.zestimate,adTargets.premieragent,adTargets.dma,adTargets.guid,adTargets.ssid,resoFacts.accessibilityFeatures,resoFacts.additionalFeeInfo,resoFacts.associationFee,resoFacts.associationAmenities,resoFacts.associationFee2,resoFacts.associationFeeIncludes,resoFacts.associationName,resoFacts.associationName2,resoFacts.associ

In [ ]:
print("Street address:", df_prop["streetAddress"].iloc[0])
print("City:", df_prop["city"].iloc[0])
print("State:", df_prop["state"].iloc[0])
print("Home status:", df_prop["homeStatus"].iloc[0])
print("Bedrooms:", df_prop["bedrooms"].iloc[0])
print("Bathrooms:", df_prop["bathrooms"].iloc[0])
print("Year built:", df_prop["yearBuilt"].iloc[0])
print("Zestimate: ${:,}".format(df_prop["zestimate"].iloc[0]))
print("Rent zestimate: ${:,}".format(df_prop["rentZestimate"].iloc[0]))
print("Description:")
df_prop["description"].iloc[0]

Street address: 3611 Potter St
City: Tampa
State: FL
Home status: SOLD
Bedrooms: 3
Bathrooms: 2
Year built: 2020
Zestimate: $353,800
Rent zestimate: $2,499
Description:


'WOW! What an amazing NEW BUILD with 3 bedrooms, 2 bathrooms, 1 car garage, fully fenced backyard and an incredible kitchen. Fall in love with the gorgeous landscaping in your front yard and a front porch with exterior lighting. As soon as you go inside immediately cherish the openness and TILED great room with your living room to your right, kitchen in the middle including GRANITE COUNTERTOPS, real wood cabinets, WALK-IN PANTRY with wire shelving racks and STAINLESS STEEL APPLIANCES. As you go farther inside, to your left you can find the TILED dining room with BLINDS and a new light fixture. Following the TILE FLOOR into the hallway on the right appreciate your washer and dryer in a closet, tucked away until it’s laundry day. As you keep going through the hallway you can find the CARPETED master bedroom with a WALK IN CLOSET, blinds and on the side is your TILED master bathroom with DUAL SINKS, granite countertops, real wood cabinets, and a stand up shower. As you exit the master bed

#### Comparables

In [ ]:
# view comparables
df_c = pd.DataFrame(df_prop["comps"].iloc[0])
df_c

""


#### Tax History

In [ ]:
# view tax history
df_tax_hist = pd.DataFrame(df_prop["taxHistory"].iloc[0])
df_tax_hist

,time,taxPaid,taxIncreaseRate,value,valueIncreaseRate
0,1649515893126,6270.88,7.952900,315812,7.888101
1,1617979893126,700.43,1.694998,35532,2.991014
2,1586443893126,259.90,0.029225,8903,0.099951
3,1554821493126,252.52,0.000000,8094,-0.509365
4,1523285493126,252.52,0.212290,16497,1.166667
5,1491749493126,208.30,0.450759,7614,0.252097
6,1460213493126,143.58,0.065055,6081,0.100036
7,1428591093126,134.81,0.212975,5528,0.100100
8,1397055093126,111.14,0.000000,5025,0.100044
9,1365519093126,111.14,0.129357,4568,0.000000


#### Price History

In [ ]:
# view price history
df_price_hist = pd.DataFrame(df_prop["priceHistory"].iloc[0])
df_price_hist

,date,time,price,pricePerSquareFoot,priceChangeRate,event,source,buyerAgent,sellerAgent,showCountyLink,postingIsRental,attributeSource
0,2023-04-08,1680912000000,340000.0,209.0,-0.028571,Sold,Stellar MLS / MFRMLS,"{'photo': None, 'profileUrl': '/profile/erica7...",{'photo': {'url': 'https://photos.zillowstatic...,False,False,"{'infoString1': 'T3424788', 'infoString2': 'St..."
1,2023-03-08,1678233600000,350000.0,215.0,0.000000,Pending sale,Stellar MLS / MFRMLS,None,None,False,False,"{'infoString1': 'T3424788', 'infoString2': 'St..."
2,2023-02-04,1675468800000,350000.0,215.0,-0.041096,Price change,Stellar MLS / MFRMLS,None,None,False,False,"{'infoString1': 'T3424788', 'infoString2': 'St..."
3,2023-01-27,1674777600000,365000.0,224.0,0.383362,Listed for sale,Stellar MLS / MFRMLS,None,None,False,False,"{'infoString1': 'T3424788', 'infoString2': 'St..."
4,2023-01-02,1672617600000,NaN,NaN,0.000000,Listing removed,Zillow Rentals,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."
5,2022-12-05,1670198400000,2400.0,1.0,0.200000,Listed for rent,Zillow Rental Manager,None,None,False,True,"{'infoString1': '', 'infoString2': 'Zillow Ren..."
6,2021-12-16,1639612800000,NaN,NaN,0.000000,Listing removed,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."
7,2021-12-14,1639440000000,2000.0,1.0,-0.047619,Price change,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': '', 'infoString2': 'Zillow Ren..."
8,2021-12-06,1638748800000,2100.0,1.0,-0.045455,Listed for rent,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': '', 'infoString2': 'Zillow Ren..."
9,2021-10-15,1634256000000,NaN,NaN,0.000000,Listing removed,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."


### <font color="green">4. Locate ZPID by Address</font>

In [ ]:
street = "12315 N Oakleaf Ave"
city = "Tampa"
state = "FL"
# zip_code = "33612" #[optional]

zpid_response = get_zpid(api_key, street, city, state, zip_code=None)

In [ ]:
zpid_response.json()["is_success"]

True

In [ ]:
# view response, top possible zpids
pd.DataFrame(zpid_response.json()["data"])

,zpid,url
0,44905318,https://www.zillow.com/homedetails/12315-N-Oak...
1,44905319,https://www.zillow.com/homedetails/12319-N-Oak...
2,44905317,https://www.zillow.com/homedetails/12311-N-Oak...
3,44905148,https://www.zillow.com/homedetails/12906-Oakle...
4,44905291,https://www.zillow.com/homedetails/12407-Oakle...
5,44904817,https://www.zillow.com/homedetails/12317-N-Rom...
6,44904546,https://www.zillow.com/homedetails/12305-N-Ore...
7,44904557,https://www.zillow.com/homedetails/11814-N-Arm...
8,44904549,https://www.zillow.com/homedetails/11834-N-Arm...
9,44905180,https://www.zillow.com/homedetails/1708-N-Vill...


In [ ]:
print("Zpid:", zpid_response.json()["data"][0]["zpid"])

Zpid: 44905318


### <font color="green">5. FSBO (For Sale By Owner) Listings</font>

In [ ]:
fsbo_url = "https://www.zillow.com/tampa-fl/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.85425150830079%2C%22east%22%3A-82.05430949169923%2C%22south%22%3A27.749100096033082%2C%22north%22%3A28.239597518118032%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22price%22%3A%7B%22max%22%3A1000000%7D%2C%22mp%22%3A%7B%22max%22%3A5155%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22category%22%3A%22cat2%22%2C%22pagination%22%3A%7B%7D%7D"

In [ ]:
fspo_response = get_listings(api_key, fsbo_url)

In [ ]:
# view count of properies returned in request
num_of_properties = fspo_response.json()["data"]["categoryTotals"]["cat2"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 24


In [ ]:
# view all listings
df_fsbo_listings = pd.json_normalize(fspo_response.json()["data"]["cat2"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_fsbo_listings))
print("Number of columns:", len(df_fsbo_listings.columns))
df_fsbo_listings

Number of rows: 24
Number of columns: 72


,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,beds,baths,area,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,timeOnZillow,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBO,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,variableData.type,variableData.text,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,streetViewURL,streetViewMetadataURL,hdpData.homeInfo.unit
0,45082799,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/0db485d6788...,True,/homedetails/6314-N-22nd-St-Tampa-FL-33610/450...,FOR_SALE,For sale by owner,"$350,000",$350K,"6314 N 22nd St, Tampa, FL 33610",3,2.0,1488,False,False,ForSale,For Sale By Owner,False,False,False,False,False,False,False,,False,False,1872050000,28.004848,-82.435290,45082799,6314 N 22nd St,33610,Tampa,FL,28.004848,-82.435290,350000.0,2.0,3.0,1488.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,366600,2299,True,False,False,FOR_SALE,350000.0,1872050000,True,False,False,USD,USA,312330.0,0.300000,acres,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45093470,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/47bcdb79dd8...,True,/homedetails/112-W-Hilda-St-Tampa-FL-33603/450...,FOR_SALE,For sale by owner,"$485,000",$485K,"112 W Hilda St, Tampa, FL 33603",3,3.0,1800,False,False,ForSale,For Sale By Owner,False,False,False,False,False,False,False,,False,False,1190150000,27.987259,-82.460686,45093470,112 W Hilda St,33603,Tampa,FL,27.987259,-82.460686,485000.0,3.0,3.0,1800.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,413100,2751,True,False,False,FOR_SALE,485000.0,1190150000,True,False,False,USD,USA,323929.0,5227.000000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,45047985,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/839dab0cd87...,True,/homedetails/4017-W-North-B-St-Tampa-FL-33609/...,FOR_SALE,For sale by owner,"$405,000",$405K,"4017 W North B St, Tampa, FL 33609",2,1.0,1009,False,False,ForSale,For Sale By Owner,False,False,False,False,False,False,False,,False,False,236330000,27.946526,-82.511410,45047985,4017 W North B St,33609,Tampa,FL,27.946526,-82.511410,405000.0,1.0,2.0,1009.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,370600,2156,True,False,False,FOR_SALE,405000.0,236330000,True,False,False,USD,USA,240969.0,5662.000000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,45064018,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/7ca276a8f76...,True,/homedetails/4729-W-Lawn-Ave-Tampa-FL-33611/45...,FOR_SALE,For sale by owner,"$389,750",$390K,"4729 W Lawn Ave, Tampa, FL 33611",3,1.0,1061,False,False,ForSale,For Sale By Owner,False,False,False,False,False,False,False,,False,False,582950000,27.901468,-82.525300,45064018,4729 W Lawn Ave,33611,Tampa,FL,27.901468,-82.525300,389750.0,1.0,3.0,1061.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,386700,2450,True,False,False,FOR_SALE,389750.0,582950000,True,False,False,USD,USA,284942.0,7405.

In [ ]:
# view price
px.box(df_fsbo_listings, x="hdpData.homeInfo.price", title="Sales Price Histogram")

#### <font color="purple">Get Owner Phone Number</font>

In [ ]:
# select a sample fspo property from the dataframe
fsbo_single_listing = "45081822" # zpid

In [ ]:
# get property detail
fsbo_prop_detail_response = get_property_detail(api_key, fsbo_single_listing)

# view property details
df_fsbo_prop = pd.json_normalize(fsbo_prop_detail_response.json()['data'])
print("Number of rows:", len(df_fsbo_prop))
print("Number of columns:", len(df_fsbo_prop.columns))
df_fsbo_prop

Number of rows: 1
Number of columns: 596


,listingDataSource,zpid,city,state,homeStatus,isListingClaimedByCurrentSignedInUser,isCurrentSignedInAgentResponsible,bedrooms,bathrooms,price,yearBuilt,streetAddress,zipcode,isCurrentSignedInUserVerifiedOwner,regionString,propertyUpdatePageLink,moveHomeMapLocationLink,propertyEventLogLink,editPropertyHistorylink,isRentalListingOffMarket,hdpUrl,nearbyCities,nearbyNeighborhoods,country,nearbyZipcodes,abbreviatedAddress,building,isUndisclosedAddress,boroughId,providerListingID,boroughSearchUrl,communityUrl,isPremierBuilder,isZillowOwned,homeType,currency,listPriceLow,livingArea,livingAreaValue,zestimate,newConstructionType,zestimateLowPercent,zestimateHighPercent,rentZestimate,restimateLowPercent,restimateHighPercent,schools,homeValues,nearbyHomes,countyFIPS,parcelId,taxHistory,priceHistory,comps,description,whatILove,contingentListingType,timeOnZillow,pageViewCount,favoriteCount,daysOnZillow,latitude,longitude,openHouseSchedule,desktopWebHdpImageLink,brokerageName,timeZone,pals,listingAccountUserId,homeInsights,sellingSoon,isIncomeRestricted,brokerId,ssid,monthlyHoaFee,propertyTaxRate,hiResImageLink,hdpTypeDimension,mlsid,propertyTypeDimension,mediumImageLink,enhancedBrokerImageUrl,responsivePhotos,buildingId,virtualTourUrl,hasApprovedThirdPartyVirtualTourUrl,photoCount,livingAreaUnits,lotSize,lotAreaValue,lotAreaUnits,postingProductType,marketingName,richMedia,cityId,stateId,zipPlusFour,numberOfUnitsTotal,foreclosureDefaultFilingDate,foreclosureAuctionFilingDate,foreclosureLoanDate,foreclosureLoanOriginator,foreclosureLoanAmount,foreclosurePriorSaleDate,foreclosurePriorSaleAmount,foreclosureBalanceReportingDate,foreclosurePastDueBalance,foreclosureUnpaidBalance,foreclosureAuctionTime,foreclosureAuctionDescription,foreclosureAuctionCity,foreclosureAuctionLocation,foreclosureDate,foreclosureAmount,foreclosingBank,foreclosureJudicialType,datePostedString,foreclosureMoreInfo,hasBadGeocode,streetViewMetadataUrlMediaWallLatLong,streetViewMetadataUrlMediaWallAddress,streetViewTileImageUrlMediumLatLong,streetViewTileImageUrlMediumAddress,streetViewServiceUrl,postingUrl,hasPublicVideo,primaryPublicVideo,richMediaVideos,photos,tourViewCount,listingAccount,listingFeedID,livingAreaUnitsShort,priceChange,priceChangeDate,priceChangeDateString,hideZestimate,comingSoonOnMarketDate,isPreforeclosureAuction,lastSoldPrice,isHousingConnector,responsivePhotosOriginalRatio,streetViewMetadataUrlMapLightboxAddress,thumb,isRecentStatusChange,isNonOwnerOccupied,county,isFeatured,rentalApplicationsAcceptedType,listingTypeDimension,featuredListingTypeDimension,brokerIdDimension,keystoneHomeStatus,pageUrlFragment,isRentalsLeadCapMet,isPaidMultiFamilyBrokerId,address.streetAddress,address.city,address.state,address.zipcode,address.neighborhood,address.community,address.subdivision,listing_sub_type.is_newHome,listing_sub_type.is_FSBO,listing_sub_type.is_FSBA,listing_sub_type.is_foreclosure,listing_sub_type.is_bankOwned,listing_sub_type.is_forAuction,listing_sub_type.is_comingSoon,listing_sub_type.is_pending,listing_sub_type.is_openHouse,citySearchUrl.text,citySearchUrl.path,zipcodeSearchUrl.path,apartmentsForRentInZipcodeSearchUrl.path,housesForRentInZipcodeSearchUrl.path,neighborhoodRegion.name,neighborhoodSearchUrl.path,stateSearchUrl.path,countySearchUrl.text,countySearchUrl.path,adTargets.zip,adTargets.aamgnrc1,adTargets.mlat,adTargets.bd,adTargets.zusr,adTargets.city,adTargets.proptp,adTargets.pid,adTargets.price_band,adTargets.yrblt,adTargets.zestimate,adTargets.lot,adTargets.zestibuck,adTargets.tflag,adTargets.listtp,adTargets.premieragent,adTargets.sqftrange,adTargets.price,adTargets.sqft,adTargets.dma,adTargets.guid,adTargets.state,adTargets.mlong,adTargets.cnty,adTargets.ssid,resoFacts.accessibilityFeatures,resoFacts.additionalFeeInfo,resoFacts.associationFee,resoFacts.associationAmenities,resoFacts.associationFee2,resoFacts.associationFeeIncludes,resoFacts.associationName,resoFacts.associationName2,resoFacts.associationPhone,resoFacts.associationP

In [ ]:
# features
# phone number
df_fsbo_prop["phone_number"] = df_fsbo_prop.apply(lambda x:
  x["contactFormRenderData.data.region_phone_number.areacode"] + '-' +
  x["contactFormRenderData.data.region_phone_number.prefix"] + '-' +
  x["contactFormRenderData.data.region_phone_number.number"], axis=1)
# full address
df_fsbo_prop["full_address"] = df_fsbo_prop.apply(lambda x:
  x["streetAddress"] + ', ' +
  x["city"] + ', ' +
  x["state"] + ' ' + x["zipcode"], axis=1)

print("The phone number for property address {} is {}".format(
    list(df_fsbo_prop["full_address"])[0],
    list(df_fsbo_prop["phone_number"])[0]))

The phone number for property address 2014 E Broad St, Tampa, FL 33610 is 813-213-3084


### <font color="green">6. Canada Listings</font>
*Coming soon*

# End Notebook